In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq

model = ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002372B4E4290>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002372B824690>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, my name is Divya!")])


AIMessage(content="Hello Divya, it's nice to meet you! 👋\n\nWhat can I do for you today? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 17, 'total_tokens': 43, 'completion_time': 0.047272727, 'prompt_time': 0.001900629, 'queue_time': 0.15909208100000002, 'total_time': 0.049173356}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--ddb9e9f2-e66f-4187-a1d5-4518cc0028cf-0', usage_metadata={'input_tokens': 17, 'output_tokens': 26, 'total_tokens': 43})

In [9]:
from langchain_core.messages import AIMessage, HumanMessage

model.invoke(
    [
        HumanMessage(content="Hi, my name is Harshit and I'm GenAI mentor!"),
        AIMessage(content="Hi Harshit, it's nice to meet you! 🔥 What can I do for you today? 😊"),
        HumanMessage(content="Hello Again! Do you remember my name and what I do professionally?"),
    ]
)


AIMessage(content="You got it! Hello Harshit, I remember you're a GenAI mentor.  \n\nIs there anything I can help you with today regarding your work with GenAI? 🤔 Perhaps you have a question about a specific model, need help brainstorming training data, or want to discuss ethical considerations in AI?  \n\nI'm here to assist however I can! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 70, 'total_tokens': 148, 'completion_time': 0.141818182, 'prompt_time': 0.004112376, 'queue_time': 0.021007123, 'total_time': 0.145930558}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--ea923b3b-42ff-4dba-9c9d-c4ec8a96f150-0', usage_metadata={'input_tokens': 70, 'output_tokens': 78, 'total_tokens': 148})

### Message History 
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore, Future interactions will then load those messages and pass them into the chain as part of the input.

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [12]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model, get_session_history)

In [13]:
config = {"configurable":{"session_id": "chat1"}}

In [14]:
response= with_message_history.invoke(
    [
        HumanMessage(content="Hi, my name is Divya and I.m a student"),
    ],
    config=config
)

print(response.content)

Hello Divya, it's nice to meet you!

I'm glad you reached out. 

What can I do for you today? Are you working on a school project, or just curious to chat? 😊  




In [16]:
retrieve1 = with_message_history.invoke(
    [HumanMessage(content="Do you remember my name?")],
    config=config
)

In [19]:
print(retrieve1)

content="Yes, I do!  I remember you're Divya. 😊 \n\nIs there anything I can help you with, Divya?  Perhaps you have a question about something you're learning in school? 📚  Or maybe you just want to talk about your day?  I'm here to listen.  👂\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 86, 'total_tokens': 157, 'completion_time': 0.129090909, 'prompt_time': 0.005916421, 'queue_time': 0.14164454699999998, 'total_time': 0.13500733}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run--1693bad1-9c12-4205-8e5e-3df026c82280-0' usage_metadata={'input_tokens': 86, 'output_tokens': 71, 'total_tokens': 157}


In [21]:
retrieve2 = with_message_history.invoke(
    [HumanMessage(content="Do you remember my profession?")],
    config=config
)

In [22]:
print(retrieve2)

content="As I said before, I don't have memory of past conversations.  \n\nSo, I don't remember you telling me your profession.  If you'd like to tell me, I'm all ears!  👂  \n\n\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 245, 'total_tokens': 298, 'completion_time': 0.096363636, 'prompt_time': 0.015127636, 'queue_time': 0.234689066, 'total_time': 0.111491272}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run--ce09b796-8b14-463b-8e7b-44140f0d3578-0' usage_metadata={'input_tokens': 245, 'output_tokens': 53, 'total_tokens': 298}


In [23]:
config2 = {"configurable": {"session_id":"chat2"}}
new_retrieve = with_message_history.invoke(
    [HumanMessage(content="Do you remember my name?")],
    config=config2
)
print(new_retrieve)

content="As a large language model, I have no memory of past conversations. If you'd like to tell me your name, I'd be happy to remember it for the duration of our current conversation.\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 15, 'total_tokens': 59, 'completion_time': 0.08, 'prompt_time': 0.001905847, 'queue_time': 0.016551642000000002, 'total_time': 0.081905847}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run--47433f11-5157-4d91-954a-3665d13a3af6-0' usage_metadata={'input_tokens': 15, 'output_tokens': 44, 'total_tokens': 59}
